In [2]:
from Data_Generation_2 import data_sets
import ZKIPModel_EM
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from math import lgamma
import warnings
from functools import lru_cache
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, r2_score
import ZkICMP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import poisson
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_poisson_deviance, mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import poisson

In [3]:
def evaluate_ZKIPModel_EM(model, X_train, X_test, y_train, y_test, true_params=None, plot_name=None, pi1=0.4, pi2=0.4):
    if plot_name is  None:
        plot_name=f'{int(X_train.shape[0]/80*100)}, {X_train.shape[1]}_({pi1:.2f},{pi2:.2f})'
    
    print("-" * 30)
    print(f"ZKIPModel_EM_({plot_name})")
    print("-" * 30)
    """Evaluate ZKIP model performance"""
    # Predict expected values
    model.fit(X_train, y_train, verbose=True)
    y_pred_expected = model.predict_expected(X_test)
    y_pred_mode = model.predict_mode(X_test)
    
    # Calculate metrics for expected value predictions
    mse_expected = mean_squared_error(y_test, y_pred_expected)
    mae_expected = mean_absolute_error(y_test, y_pred_expected)
    r2 = r2_score(y_test, y_pred_expected)
    mse_mode = mean_squared_error(y_test, y_pred_mode)
    mae_mode = mean_absolute_error(y_test, y_pred_mode)
    r2_mode = r2_score(y_test, y_pred_mode)
    
    # For mode predictions, we need to check accuracy
    accuracy_mode = np.mean(y_test == y_pred_mode)
    print("Performance metrics:")
    print(f"  Expected value prediction:")
    print(f"    MSE: {mse_expected:.4f}")
    print(f"    MAE: {mae_expected:.4f}")
    print(f"    r^2: {r2:.4f}")
    print(f"  Mode prediction:")
    print(f"    MSE: {mse_mode:.4f}")
    print(f"    MAE: {mse_mode:.4f}")
    print(f"    r^2: {r2_mode:.4f}")
    print(f"    Accuracy: {accuracy_mode:.4f}")
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    # Plot 1: True vs predicted (expected)
    axes[0, 0].scatter(y_test, y_pred_expected, alpha=0.6)
    axes[0, 0].plot([0, max(y_test)], [0, max(y_test)], 'r--', alpha=0.8)
    axes[0, 0].set_xlabel('True Count')
    axes[0, 0].set_ylabel('Predicted Expected Count')
    axes[0, 0].set_title('True vs Predicted (Expected Value)')
    axes[0, 0].grid(True, alpha=0.3)
    # Plot 2: True vs predicted (mode)
    axes[0, 1].scatter(y_test, y_pred_mode, alpha=0.6)
    axes[0, 1].plot([0, max(y_test)], [0, max(y_test)], 'r--', alpha=0.8)
    axes[0, 1].set_xlabel('True Count')
    axes[0, 1].set_ylabel('Predicted Mode')
    axes[0, 1].set_title('True vs Predicted (Mode)')
    axes[0, 1].grid(True, alpha=0.3)
    # Plot 3: Residuals
    residuals = y_test - y_pred_expected
    axes[1, 0].scatter(y_pred_expected, residuals, alpha=0.6)
    axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.8)
    axes[1, 0].set_xlabel('Predicted Expected Count')
    axes[1, 0].set_ylabel('Residuals')
    axes[1, 0].set_title('Residual Plot')
    axes[1, 0].grid(True, alpha=0.3)
    # Plot 4: Count distribution comparison
    axes[1, 1].hist(y_test, bins=range(0, int(max(y_test)) + 2), alpha=0.7, label='True', density=True)
    axes[1, 1].hist(y_pred_mode, bins=range(0, int(max(y_test)) + 2), alpha=0.7, label='Predicted Mode', density=True)
    axes[1, 1].set_xlabel('Count')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Count Distribution: True vs Predicted')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'Plots/ZKIPModel_EM_({plot_name}).png', dpi=300, bbox_inches='tight')
    plt.close()
    #plt.show()

In [4]:
def evaluate_ZkICMP(model, X_train, X_test, y_train, y_test, plot_name=None, pi1=0.4, pi2=0.4):
    if plot_name is  None:
        plot_name=f'{int(X_train.shape[0]/80*100)}, {X_train.shape[1]}_({pi1:.2f},{pi2:.2f})'

    print("-" * 30)
    print(f"ZkICMP_({plot_name})")
    print("-" * 30)
    model.fit(X_train, y_train, verbose=True)
    y_pred_counts, y_pred_probs, y_pred_rates, count_values = model.predict(X_test)
    
    # Calculate metrics
    mse_rates = mean_squared_error(y_test, y_pred_rates)
    mae_rates = mean_absolute_error(y_test, y_pred_rates)
    r2_rates = r2_score(y_test, y_pred_rates)
    mse_counts = mean_squared_error(y_test, y_pred_counts)
    mae_counts = mean_absolute_error(y_test, y_pred_counts)
    r2_counts = r2_score(y_test, y_pred_counts)
    accuracy = accuracy_score(y_test, y_pred_counts)
    
    print("Performance metrics:")
    print(f"  Rate predictions (λ):")
    print(f"    MSE: {mse_rates:.4f}")
    print(f"    MAE: {mae_rates:.4f}")
    print(f"    r^2: {r2_rates:.4f}")

    print(f"  Count predictions:")
    print(f"    MSE: {mse_counts:.4f}")
    print(f"    MAE: {mae_counts:.4f}")
    print(f"    r^2: {r2_counts:.4f}")
    print(f"    Accuracy: {accuracy:.4f}")

    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # Plot 1: True vs predicted rates
    axes[0, 0].scatter(y_test, y_pred_rates, alpha=0.6)
    max_val = max(np.max(y_test), np.max(y_pred_rates))
    axes[0, 0].plot([0, max_val], [0, max_val], 'r--', alpha=0.8)
    axes[0, 0].set_xlabel('True Count')
    axes[0, 0].set_ylabel('Predicted Rate (λ)')
    axes[0, 0].set_title('True Count vs Predicted CMP Rate')
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: True vs predicted counts
    axes[0, 1].scatter(y_test, y_pred_counts, alpha=0.6)
    max_val = max(np.max(y_test), np.max(y_pred_counts))
    axes[0, 1].plot([0, max_val], [0, max_val], 'r--', alpha=0.8)
    axes[0, 1].set_xlabel('True Count')
    axes[0, 1].set_ylabel('Predicted Count')
    axes[0, 1].set_title('True vs Predicted Counts')
    axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: Residuals
    residuals = y_test - y_pred_rates
    axes[1, 0].scatter(y_pred_rates, residuals, alpha=0.6)
    axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.8)
    axes[1, 0].set_xlabel('Predicted Rate (λ)')
    axes[1, 0].set_ylabel('Residuals')
    axes[1, 0].set_title('Residual Plot')
    axes[1, 0].grid(True, alpha=0.3)

    # Plot 4: Count distribution comparison
    max_bin = int(max(np.max(y_test), np.max(y_pred_counts))) + 2
    bins = range(0, min(max_bin, 50))
    axes[1, 1].hist(y_test, bins=bins, alpha=0.7, label='True', density=True)
    axes[1, 1].hist(y_pred_counts, bins=bins, alpha=0.7, label='Predicted', density=True)
    axes[1, 1].set_xlabel('Count')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Count Distribution: True vs Predicted')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'Plots/ZKICMP_({plot_name}).png', dpi=300, bbox_inches='tight')
    plt.close()
    #plt.show()

In [5]:
def evaluate_HistGradientBoostingRegressor(model, X_train, y_train, X_test, y_test, plot_name=None, pi1=0.4, pi2=0.4):
    if plot_name is  None:
        plot_name=f'{int(X_train.shape[0]/80*100)}, {X_train.shape[1]}_({pi1:.2f},{pi2:.2f})'

    print("-" * 30)
    print(f"HistGradientBoostingRegressor_({plot_name})")
    print("-" * 30)
    model.fit(X_train, y_train.ravel())
    
    # Predict expected values (Poisson rates λ)
    y_pred_rates = model.predict(X_test)

    # Convert rates to integer counts (by rounding)
    y_pred_counts = np.round(y_pred_rates).astype(int)

    # Calculate metrics for rate predictions
    mse_rates = mean_squared_error(y_test, y_pred_rates)
    mae_rates = mean_absolute_error(y_test, y_pred_rates)
    r2_rates = r2_score(y_test, y_pred_rates)

    # Metrics for count predictions
    mse_counts = mean_squared_error(y_test, y_pred_counts)
    mae_counts = mean_absolute_error(y_test, y_pred_counts)
    r2_counts = r2_score(y_test, y_pred_counts)

    # Poisson-specific metrics
    def poisson_deviance(y_true, y_pred):
        """Poisson deviance - better for count data than MSE"""
        # Avoid log(0) and division by zero issues
        mask = (y_true > 0) & (y_pred > 0)
        if np.any(mask):
            deviance = 2 * np.mean(y_true[mask] * np.log(y_true[mask] / y_pred[mask]) - (y_true[mask] - y_pred[mask]))
        else:
            # Fallback to MSE if no valid pairs
            deviance = np.mean((y_true - y_pred) ** 2)
        return deviance

    deviance = poisson_deviance(y_test, y_pred_rates)

    print("Performance metrics:")
    print(f"  Rate predictions (λ):")
    print(f"    MSE: {mse_rates:.4f}")
    print(f"    MAE: {mae_rates:.4f}")
    print(f"    R²: {r2_rates:.4f}")

    print(f"  Count predictions (rounded):")
    print(f"    MSE: {mse_counts:.4f}")
    print(f"    MAE: {mae_counts:.4f}")
    print(f"    R²: {r2_counts:.4f}")
    accuracy = (y_pred_counts==y_test).sum()/(y_pred_counts!=y_test).sum()
    print(f"    Accuracy: {accuracy:.4f}")

    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # Plot 1: True vs predicted rates
    axes[0, 0].scatter(y_test, y_pred_rates, alpha=0.6)
    max_val = max(np.max(y_test), np.max(y_pred_rates))
    axes[0, 0].plot([0, max_val], [0, max_val], 'r--', alpha=0.8)
    axes[0, 0].set_xlabel('True Count')
    axes[0, 0].set_ylabel('Predicted Rate (λ)')
    axes[0, 0].set_title('True Count vs Predicted Poisson Rate')
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: True vs predicted counts
    axes[0, 1].scatter(y_test, y_pred_counts, alpha=0.6)
    max_val = max(np.max(y_test), np.max(y_pred_counts))
    axes[0, 1].plot([0, max_val], [0, max_val], 'r--', alpha=0.8)
    axes[0, 1].set_xlabel('True Count')
    axes[0, 1].set_ylabel('Predicted Count (Rounded)')
    axes[0, 1].set_title('True vs Predicted Counts')
    axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: Residuals
    residuals = y_test - y_pred_rates
    axes[1, 0].scatter(y_pred_rates, residuals, alpha=0.6)
    axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.8)
    axes[1, 0].set_xlabel('Predicted Rate (λ)')
    axes[1, 0].set_ylabel('Residuals')
    axes[1, 0].set_title('Residual Plot')
    axes[1, 0].grid(True, alpha=0.3)

    # Plot 4: Count distribution comparison
    max_bin = int(max(np.max(y_test), np.max(y_pred_counts))) + 2
    bins = range(0, min(max_bin, 50))  # Limit bins to avoid too many
    axes[1, 1].hist(y_test, bins=bins, alpha=0.7, label='True', density=True)
    axes[1, 1].hist(y_pred_counts, bins=bins, alpha=0.7, label='Predicted', density=True)
    axes[1, 1].set_xlabel('Count')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Count Distribution: True vs Predicted')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    
    plt.tight_layout()
    plt.savefig(f'Plots/HistGradientBoostingRegressor_({plot_name}).png', dpi=300, bbox_inches='tight')
    plt.close()
    #plt.show()

In [6]:
def evaluate_rf(rf, X_train, X_test, y_train, y_test, plot_name=None, pi1=0.4, pi2=0.4):
    if plot_name is  None:
        plot_name=f'({int(X_train.shape[0]/80*100)}, {X_train.shape[1]})_({pi1:.2f},{pi2:.2f})'
    print("-" * 30)
    print(f"PoissonRandomForestRegressor_{plot_name}")
    print("-" * 30)
    rf.fit(X_train, y_train)
    # Get rate predictions (λ)
    y_pred_rates = rf.predict(X_test)

    # Get count predictions (rounded)
    y_pred_counts = np.round(y_pred_rates).astype(int)

    # Calculate metrics for rates
    mse_rates = mean_squared_error(y_test, y_pred_rates)
    mae_rates = mean_absolute_error(y_test, y_pred_rates)
    r2_rates = r2_score(y_test, y_pred_rates)
    deviance = mean_poisson_deviance(y_test, y_pred_rates)

    # Calculate metrics for counts
    mse_counts = mean_squared_error(y_test, y_pred_counts)
    mae_counts = mean_absolute_error(y_test, y_pred_counts)
    r2_counts = r2_score(y_test, y_pred_counts)
    
    print("Performance metrics:")
    print(f"  Rate predictions (λ):")
    print(f"    MSE: {mse_rates:.4f}")
    print(f"    MAE: {mae_rates:.4f}")
    print(f"    R²: {r2_rates:.4f}")
    
    print(f"  Count predictions (rounded):")
    print(f"    MSE: {mse_counts:.4f}")
    print(f"    MAE: {mae_counts:.4f}")
    print(f"    R²: {r2_counts:.4f}")
    accuracy = (y_pred_counts == y_test).sum() / len(y_test) 
    print(f"    Accuracy: {accuracy:.4f}")

    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # Plot 1: True vs predicted rates
    axes[0, 0].scatter(y_test, y_pred_rates, alpha=0.6)
    max_val = max(np.max(y_test), np.max(y_pred_rates))
    axes[0, 0].plot([0, max_val], [0, max_val], 'r--', alpha=0.8)
    axes[0, 0].set_xlabel('True Count')
    axes[0, 0].set_ylabel('Predicted Rate (λ)')
    axes[0, 0].set_title('True Count vs Predicted Poisson Rate')
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: True vs predicted counts
    axes[0, 1].scatter(y_test, y_pred_counts, alpha=0.6)
    max_val = max(np.max(y_test), np.max(y_pred_counts))
    axes[0, 1].plot([0, max_val], [0, max_val], 'r--', alpha=0.8)
    axes[0, 1].set_xlabel('True Count')
    axes[0, 1].set_ylabel('Predicted Count (Rounded)')
    axes[0, 1].set_title('True vs Predicted Counts')
    axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: Residuals
    residuals = y_test - y_pred_rates
    axes[1, 0].scatter(y_pred_rates, residuals, alpha=0.6)
    axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.8)
    axes[1, 0].set_xlabel('Predicted Rate (λ)')
    axes[1, 0].set_ylabel('Residuals')
    axes[1, 0].set_title('Residual Plot')
    axes[1, 0].grid(True, alpha=0.3)

    # Plot 4: Count distribution comparison
    max_bin = int(max(np.max(y_test), np.max(y_pred_counts))) + 2
    bins = range(0, min(max_bin, 50))  # Limit bins to avoid too many
    axes[1, 1].hist(y_test, bins=bins, alpha=0.7, label='True', density=True)
    axes[1, 1].hist(y_pred_counts, bins=bins, alpha=0.7, label='Predicted', density=True)
    axes[1, 1].set_xlabel('Count')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Count Distribution: True vs Predicted')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    if plot_name is  None:
        plot_name=f'{int(X_train.shape[0]/80*100)}, {X_train.shape[1]}'
    plt.tight_layout()
    plt.savefig(f'Plots/PoissonRandomForestRegressor_({plot_name}).png', dpi=300, bbox_inches='tight')
    plt.close()
    #plt.show()

In [7]:
def fit_all(n, pi1, pi2):
    data_sets(n=n, k_inflated=3, true_beta=None, true_gamma=None, true_delta=None, 
                      pi1=pi1, pi2=pi1, pi3=None, seed=42)
    
    X_train_name=f'DataSets/X_train_0k_inflated_({n}, 2)_({pi1:.2f},{pi2:.2f}).csv'
    X_test_name=f'DataSets/X_test_0k_inflated_({n}, 2)_({pi1:.2f},{pi2:.2f}).csv'
    y_train_df_name=f'DataSets/y_train ({n}, 2)_({pi1:.2f},{pi2:.2f}).csv'
    y_test_df_name=f'DataSets/y_test ({n}, 2)_({pi1:.2f},{pi2:.2f}).csv'
    
    X_train = pd.read_csv(X_train_name)
    X_test = pd.read_csv(X_test_name)
    y_train = pd.read_csv(y_train_df_name)
    y_test = pd.read_csv(y_test_df_name)
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)
    # Convert to numpy arrays for ZkICMP
    X_train = X_train.values
    X_test = X_test.values
    
    # ZKIPModel_EM
    model = ZKIPModel_EM.ZKIPModel_EM(k_inflated=3) 
    evaluate_ZKIPModel_EM(model, X_train, X_test, y_train, y_test, pi1=pi1, pi2=pi2)
    
    # ZkICMP
    model = ZkICMP.ZkICMP(k=3)
    evaluate_ZkICMP(model, X_train, X_test, y_train, y_test, pi1=pi1, pi2=pi2)
    
    # HistGradientBoostingRegressor
    poisson_GB = HistGradientBoostingRegressor(
            loss='poisson',        # This is the key - uses Poisson likelihood
            random_state=42,
            max_iter=100,
            learning_rate=0.1,
            max_depth=6,
            min_samples_leaf=20    # Good for count data to prevent overfitting
        )
    evaluate_HistGradientBoostingRegressor(poisson_GB, X_train, y_train, X_test, y_test, pi1=pi1, pi2=pi2)
    
    # RandomForestRegressor
    rf = RandomForestRegressor(
        n_estimators=300,
        criterion="poisson",
        min_samples_leaf=5,
        n_jobs=-1,
        random_state=42
    )
    
    evaluate_rf(rf, X_train ,X_test, y_train, y_test, pi1=pi1, pi2=pi2)

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
%%capture output
n=100
pi1=0.2
pi2=0.2
print(f'n={n}_pi1={pi1}_pi2-{pi2}')

fit_all(n, pi1, pi2)
with open(f'n={n}_pi1={pi1}_pi2={pi2}.txt', 'w', encoding='utf-8') as f:
    f.write(output.stdout)


In [12]:
import io
from contextlib import redirect_stdout

nn = [50, 100, 200, 500, 1000, 2000, 5000, 10000]
pi1 = pi2 = 0.4

for n in nn:
    buffer = io.StringIO()

    with redirect_stdout(buffer):
        print(f'n={n}_pi1={pi1}_pi2={pi2}')
        fit_all(n, pi1, pi2)

    # Get only this iteration's output
    iteration_output = buffer.getvalue()

    # Save to a file
    with open(f'results_txt/n={n}_pi1={pi1}_pi2={pi2}.txt', 'w', encoding='utf-8') as f:
        f.write(iteration_output)
